## 1. Setting Up the Environment
Setting environment, including installing necessary libraries and importing dataset

In [1]:
# Importing Libraries
import os
import timeit
import random
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import h5py
import obspy
from obspy import Trace, UTCDateTime, read

import keras
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from tensorflow.keras import layers
from sklearn.metrics import f1_score, accuracy_score

In [2]:
# Setting Dataset Paths: Metadata
csv_path = 'dataset\\stead\\metadata.csv'

# Read metadata, fill noise data p-arrival value with 0
df = pd.read_csv(csv_path)
df['p_arrival_sample'] = df['p_arrival_sample'].fillna(0)
df

C:\Users\pc baru\AppData\Local\Temp\ipykernel_10632\3882101547.py:6: DtypeWarning: Columns (7,11,13,14,24,25,26,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
0,TA,109C,HH,32.8889,-117.1051,150.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-21 05:55:00,noise,109C.TA_201510210555_NO
1,TA,109C,HH,32.8889,-117.1051,150.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-06 14:50:00,noise,109C.TA_201511061450_NO
2,TA,109C,HH,32.8889,-117.1051,150.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-07 02:20:00,noise,109C.TA_201511070220_NO
3,TA,109C,HH,32.8889,-117.1051,150.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-14 05:15:00,noise,109C.TA_201511140515_NO
4,TA,109C,HH,32.8889,-117.1051,150.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-25 18:50:00,noise,109C.TA_201512251850_NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265652,WY,YHL,HH,44.8509,-111.1830,2691.0,400.0,manual,0.68,4.21,...,NaN,NaN,0.1253,13.93,124.9,[34.40000153 33. 36.70000076],[[1239.]],2017-06-21 09:40:43.810000,earthquake_local,YHL.WY_20170621094042_EV
1265653,WY,YHL,HH,44.8509,-111.1830,2691.0,600.0,manual,0.65,4.84,...,NaN,NaN,0.1553,17.26,121.7,[44.20000076 43.79999924 41.40000153],[[1551.]],2017-06-21 12:21:05.390000,earthquake_local,YHL.WY_20170621122104_EV
1265654,WY,YHL,HH,44.8509,-111.1830,2691.0,500.0,manual,0.65,4.82,...,NaN,NaN,0.1534,17.05,121.5,[47.5 39.90000153 42.70000076],[[1443.]],2017-06-21 12:23:40.200000,earthquake_local,YHL.WY_20170621122339_EV
1265655,WY,YHL,HH,44.8509,-111.1830,2691.0,500.0,manual,0.59,4.26,...,NaN,NaN,0.1267,14.08,126.6,[24.70000076 21.89999962 21. ],[[1322.]],2017-06-21 13:34:40.330000,earthquake_local,YHL.WY_20170621133439_EV


In [3]:
# Setting Dataset Paths: Waveform
wave_path = lambda x: f'dataset\\stead\\waveform\\chunk{x}.hdf5'

counting = 0
trace_names = []
chuck_name = []

# Read Waveform in 6 chunk
for i in tqdm(range(6)):
    with h5py.File(wave_path(i+1), "r") as f:
        counting += len(f['data'].keys())
        trace_names += list(f['data'].keys())
        chuck_name += [i+1] * len(f['data'].keys())
        
print("Total data:", counting)

chunkdict = {trc:chu for trc, chu in zip(trace_names, chuck_name)}
df['chunk'] = df['trace_name'].apply(lambda x: chunkdict[x])

# Just resample 200000 for each chunk
chunk_length = 200000
chunk_traces = pd.concat([df[df.chunk==i].iloc[:chunk_length] for i in range(1,7)])[['chunk','trace_name','p_arrival_sample']].groupby('chunk').agg(list)
chunk_traces['length'] = chunk_traces['trace_name'].apply(lambda x: len(x))
chunk_traces

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.65s/it]


Total data: 1265657


,trace_name,p_arrival_sample,length
chunk,,,
1,"[109C.TA_201510210555_NO, 109C.TA_201511061450...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1000
2,"[109C.TA_20060723155859_EV, 109C.TA_2006110315...","[700.0, 600.0, 500.0, 900.0, 700.0, 700.0, 900...",1000
3,"[B087.PB_20111103062804_EV, B087.PB_2011110306...","[599.0, 499.0, 399.0, 900.0, 600.0, 700.0, 800...",1000
4,"[CPE.AZ_20121116014416_EV, CPE.AZ_201211160937...","[400.0, 500.0, 700.0, 1000.0, 400.0, 800.0, 40...",1000
5,"[HPC.NC_20130403221010_EV, HPC.NC_201304040259...","[800.0, 500.0, 600.0, 1000.0, 700.0, 800.0, 90...",1000
6,"[PAH.NN_20151214105520_EV, PAH.NN_201512141336...","[600.0, 400.0, 500.0, 800.0, 400.0, 700.0, 700...",1000


## 2. Data Preprocessing

In [4]:
# Interpolation function
def cstream(dt0, sr1, sr2, data):
    # Set the start time, end time, and sampling rate
    start_time = UTCDateTime.now() - dt0  # Replace with your desired start time
    end_time = UTCDateTime.now()  # Calculate the end time based on the data length and sampling rate
    sampling_rate = sr1
    st = obspy.Stream()
    
    # Create an ObsPy trace
    trace0 = obspy.Trace(data=data[:,0], header={
        'starttime': start_time,
        'endtime': end_time,
        'sampling_rate': sampling_rate
    })
    trace1 = obspy.Trace(data=data[:,1], header={
        'starttime': start_time,
        'endtime': end_time,
        'sampling_rate': sampling_rate
    })
    trace2 = obspy.Trace(data=data[:,2], header={
        'starttime': start_time,
        'endtime': end_time,
        'sampling_rate': sampling_rate
    })
    st.append(trace0)
    st.append(trace1)
    st.append(trace2)
    trace = st.interpolate(sampling_rate=sr2, method="linear")
    return np.moveaxis(np.array([
        trace[-3].data[:],
        trace[-2].data[:],
        trace[-1].data[:],
        ]), -2, -1)

In [5]:
def prep(series, mode):
    if mode=="z_norm":
        mean = np.mean(series)
        std = np.std(series)
        return (series - mean) / std
    elif mode=="min_max":
        min_val=0
        max_val=1
        series_min = np.min(series)
        series_max = np.max(series)
        return (series - series_min) * (max_val - min_val) / (series_max - series_min) + min_val
    elif mode=="mean_norm":
        mean = series.mean(0)
        return series - mean
    elif mode=="max_abs":
        max_absolute = np.max(np.abs(series))
        return series / max_absolute
    elif mode=="power_transform":
        sign = np.sign(series)
        return sign * np.abs(series) ** 0.5
    elif mode=="scaled_norm": 
        return series/1000
    elif mode=="proposed_norm": 
        return (series - series[0])/1000
    return series   

In [6]:
# Create function for vectorization data and label
def set_label(p_point, window, residual, source_sampling_rate=100, max_data=60):
    return 1 if p_point>=(((window/2)-residual)*source_sampling_rate) and p_point<((max_data-(residual+(window/2)))*source_sampling_rate) else 0

def get_window(array, label, p_point, window, residual, sampling_rate, source_sampling_rate=100, max_data=60):
    # Create available area for wave_start
    available = np.arange(0, int((max_data-window)*source_sampling_rate))
    # Create wave_start that the P-wave arrival point (p_point) is centered within the window +- residual shift
    p_area = np.arange(p_point - (((window/2)+residual)*source_sampling_rate), p_point - (((window/2)-residual)*source_sampling_rate))
    if label:
        # Random choice wave_start in p-arrival area
        wave_start = np.random.choice(p_area)
        # Shifting negative value to 0 
        wave_start = int(max(wave_start, 0))
    else:
        # Random choice wave_start excluding p-wave area
        wave_start = int(np.random.choice(np.setdiff1d(available, p_area)))
    wave = array[wave_start:wave_start+int(window*source_sampling_rate)+1]
    if sampling_rate!=source_sampling_rate:
        wave = cstream(window, source_sampling_rate, sampling_rate, wave)
    return wave[:-1]

In [7]:
# Read Waveform in 6 chunk
DATA = [0,0,0,0,0,0]
for i in tqdm(range(6)):
    DATA[i] = h5py.File(wave_path(i+1), "r")

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 3008.83it/s]


In [17]:
# Create data generator
class DataGenerator(keras.utils.Sequence):
    def __init__(self, data, window, residual, sampling_rate, preprocessing='proposed_norm', batch_size=128):
        self.data = data
        self.window = window
        self.residual = residual
        self.sampling_rate = sampling_rate
        self.preprocessing = preprocessing
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = min((index + 1) * self.batch_size, len(self.data))
        X, y = [], []
        balanced_batch = np.random.binomial(n=1, p=0.5, size=self.batch_size)
        for enu,idx in enumerate(self.data[start_index:end_index]):
            name = chunk_traces.loc[1+idx//chunk_length].trace_name[idx%chunk_length]
            data = DATA[idx//chunk_length]['data'][name][:]
            p_arrival = chunk_traces.loc[1+idx//chunk_length].p_arrival_sample[idx%chunk_length]
            label = chunk_traces.loc[1+idx//chunk_length].label[idx%chunk_length] * balanced_batch[enu]
            data = get_window(data, label, p_arrival, self.window, self.residual, self.sampling_rate)
            data = prep(data, self.preprocessing)
            X.append(data)
            y.append(label)
        return np.array(X), np.array(y)

## 3. Modeling

In [24]:
# Modeling function
def evaluate_dl(name, clf, train, test, save=False):
    start_time = timeit.default_timer()
    mymodelbefore = sorted([pt.split('_')[0] for pt in os.listdir(f'{foldername}/{window}s/') if '.h5' in pt])
    if name in mymodelbefore:
        clf = keras.models.load_model(f'{foldername}/{window}s/{name}_{window}s_{sampling_rate}hz.h5')
        clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        save = False
    else:
        callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]
        clf.fit(train, validation_data=test, epochs=500, batch_size=4096, callbacks=callbacks, shuffle=True)
    pred_test = clf.predict(test)
    loss_test = round(clf.evaluate(test)[0], 5)
    acc_test = round(clf.evaluate(test)[1], 5)
    elapsed_test = timeit.default_timer()-start_time
    print(name)
    print('loss_test:',loss_test)
    print('acc_test:',acc_test)
    print('elapsed:',elapsed_test)
    print('==='*50,'\n')
    if save:
        clf.save(f'{foldername}/{window}s/{name}_{window}s_{sampling_rate}hz.h5')
    return [name, loss_test, acc_test, elapsed_test]  

def se_block(input_tensor, ratio=4):
    """The Squeeze-and-Excitation block.
    
    Args:
    input_tensor: input tensor
    ratio: reduction ratio for the squeeze operation
    
    Returns:
    output tensor after the SE block
    """
    # Squeeze operation
    res = layers.SeparableConv1D(int(input_tensor.shape[-1]*2), kernel_size=3, padding='same', activation="relu")(input_tensor)
    se_avg_pool = layers.GlobalAveragePooling1D()(res)
    
    se_dense_1 = layers.Dense(input_tensor.shape[-1] // ratio, activation='relu')(se_avg_pool)
    se_dense_2 = layers.Dense(res.shape[-1], activation='sigmoid')(se_dense_1)
    
    # Scale the input
    se_scale = layers.Multiply()([res, se_dense_2])
    proj = layers.Dense(input_tensor.shape[-1])(se_scale)
    proj = layers.Dropout(0.15)(proj)
    
    return layers.LayerNormalization(epsilon=1e-6)(proj + input_tensor) 

# Transformer components
def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.15):
    # Multi-head attention
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attn_output = layers.Dropout(rate)(attn_output)
    out1 = layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    
    # Feed-forward network
    ffn_output = layers.Dense(ff_dim, activation="linear")(out1)
    ffn_output = keras.layers.LeakyReLU(alpha=0.1)(ffn_output)
    ffn_output = layers.Dense(embed_dim)(ffn_output)
    ffn_output = layers.Dropout(rate)(ffn_output)
    return layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

def convmodel(inputs, spatialdrop=False, se=False):
    x = keras.layers.Conv1D(32, kernel_size=3, padding='same', activation="relu")(inputs)
    if spatialdrop:
        x = keras.layers.SpatialDropout1D(0.15)(x)
    x = keras.layers.MaxPooling1D(2)(x)
    x = keras.layers.Conv1D(64, kernel_size=3, padding='same', activation="relu")(x)
    if spatialdrop:
        x = keras.layers.SpatialDropout1D(0.15)(x)
    x = keras.layers.MaxPooling1D(2)(x)
    x = keras.layers.Conv1D(128, kernel_size=3, padding='same', activation="relu")(x)
    if spatialdrop:
        x = keras.layers.SpatialDropout1D(0.15)(x)
    x = keras.layers.MaxPooling1D(2)(x)
    return x
    
def DLmodel(name, input_shape): 
    inputs = keras.layers.Input(shape=input_shape)
   
    if name.split('-')[0]=='01convflat':
        x = convmodel(inputs, spatialdrop=False)
        layer = keras.layers.Flatten()(x)
    if name.split('-')[0]=='02convspat':
        x = convmodel(inputs, spatialdrop=True)
        layer = keras.layers.Flatten()(x)
    if name.split('-')[0]=='03convse':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        layer = keras.layers.Flatten()(x)
    if name.split('-')[0]=='03convse2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        layer = keras.layers.Flatten()(x)
    if name.split('-')[0]=='03convse3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        layer = keras.layers.Flatten()(x)
    
    if name.split('-')[0]=='04convset':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        for _ in range(1): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convselstm':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='04convse2t':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        for _ in range(1): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convse2lstm':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='04convse3t':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        for _ in range(1): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convse3lstm':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        layer = keras.layers.LSTM(256)(x)
        
    if name.split('-')[0]=='04convset2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        for _ in range(2): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convselstm2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='04convse2t2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        for _ in range(2): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convse2lstm2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='04convse3t2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        for _ in range(2): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convse3lstm2':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
        
    if name.split('-')[0]=='04convset3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        for _ in range(3): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convselstm3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='04convse2t3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        for _ in range(3): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convse2lstm3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='04convse3t3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        for _ in range(3): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convse3lstm3':
        x = convmodel(inputs, spatialdrop=True)
        x = se_block(x)
        x = se_block(x)
        x = se_block(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    
    if name.split('-')[0]=='06convt':
        x = convmodel(inputs, spatialdrop=True)
        for _ in range(1): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='07convlstm':
        x = convmodel(inputs, spatialdrop=True)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='06convt2':
        x = convmodel(inputs, spatialdrop=True)
        for _ in range(2): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='07convlstm2':
        x = convmodel(inputs, spatialdrop=True)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='06convt3':
        x = convmodel(inputs, spatialdrop=True)
        for _ in range(3): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='07convlstm3':
        x = convmodel(inputs, spatialdrop=True)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    
    if name.split('-')[0]=='08lstm':
        layer = keras.layers.LSTM(256)(inputs)
    if name.split('-')[0]=='08lstm2':
        x = keras.layers.LSTM(256, return_sequences=True)(inputs)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='08lstm3':
        x = keras.layers.LSTM(256, return_sequences=True)(inputs)
        x = keras.layers.LSTM(256, return_sequences=True)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='09trans':
        x = inputs
        for _ in range(1): x = transformer_encoder(x, 3, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='09trans2':
        x = inputs
        for _ in range(2): x = transformer_encoder(x, 3, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='09trans3':
        x = inputs
        for _ in range(3): x = transformer_encoder(x, 3, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    
    if name.split('-')[0]=='08lstmspat':
        x = keras.layers.LSTM(256, return_sequences=True)(inputs)
        x = keras.layers.SpatialDropout1D(0.15)(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='09transspat':
        x = inputs
        x = transformer_encoder(x, 3, 4, 128)
        x = keras.layers.SpatialDropout1D(0.15)(x)
        x = transformer_encoder(x, 3, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='03convsenospat':
        x = convmodel(inputs, spatialdrop=False)
        x = se_block(x)
        layer = keras.layers.Flatten()(x)
    if name.split('-')[0]=='04convsetnospat':
        x = convmodel(inputs, spatialdrop=False)
        x = se_block(x)
        for _ in range(1): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='05convselstmnospat':
        x = convmodel(inputs, spatialdrop=False)
        x = se_block(x)
        layer = keras.layers.LSTM(256)(x)
    if name.split('-')[0]=='06convtnospat':
        x = convmodel(inputs, spatialdrop=False)
        for _ in range(1): x = transformer_encoder(x, 128, 4, 128)
        layer = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    if name.split('-')[0]=='07convlstmnospat':
        x = convmodel(inputs, spatialdrop=False)
        layer = keras.layers.LSTM(256)(x)

    layer = keras.layers.Dense(64, activation='relu')(layer)
    layer = keras.layers.Dropout(0.5)(layer)
    outputs = keras.layers.Dense(1, activation='sigmoid')(layer)
    model = keras.Model(inputs=inputs, outputs=outputs, name=name)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

## 4. Training

In [27]:
# Hyperparameter
window = 8
sampling_rate = 20
residual = 1
batch_size = 128

# Set label for each chunk
label_chunk = [[set_label(v, window, residual) for v in chunk_traces.loc[i]['p_arrival_sample']] for i in range(1,7)]
chunk_traces['label'] = label_chunk

# Split data into training-validation-testing
train_series = pd.DataFrame({'label':label_chunk[0]+label_chunk[1]+label_chunk[2]+label_chunk[3]})
valid_series = pd.DataFrame({'label':label_chunk[4]+label_chunk[5]})
train_series = train_series.sample(train_series.shape[0])
valid_series = valid_series.sample(valid_series.shape[0])

# Get data with generator
train_gen = DataGenerator(train_series.index.values, window, residual, sampling_rate, preprocessing='proposed_norm', batch_size=batch_size)
valid_gen = DataGenerator(valid_series.index.values, window, residual, sampling_rate, preprocessing='proposed_norm', batch_size=batch_size)

In [28]:
# Create folder
# Create folder
foldername = "save_model/stead"
try:
    os.makedirs(foldername+'/'+str(window)+'s')
except:
    pass

result = []
modelnames = [
     '04convset', #'08lstmspat', '09transspat', '03convsenospat', '05convselstmnospat', '06convtnospat', '07convlstmnospat',
    ]
for name in tqdm(modelnames):
    res = evaluate_dl(name, DLmodel(name, (int(window*sampling_rate), 3)), train_gen, valid_gen, save=True)
    result.append(res)
    df_result = pd.DataFrame(result, columns=['model', 'loss', 'accuracy', 'elapsed'])
    df_result.to_csv(f'{foldername}/{window}s/result_{name}_{window}s_{sampling_rate}hz.csv', index=False)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Model: "04convset"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 160, 3)]     0                                            
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 160, 32)      320         input_7[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_18 (SpatialDr (None, 160, 32)      0           conv1d_18[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_18 (MaxPooling1D) (None, 80, 32)       0           spatial_dropout1d_18[0][0]       
__________________________________________________________________________________________

16/16 [==============================] - 5s 304ms/step - loss: 0.1194 - accuracy: 0.9595


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:34<00:00, 94.76s/it]

04convset
loss_test: 0.13689
acc_test: 0.9595
elapsed: 94.37375909999992

